In [0]:
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import year, to_date

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "#####",
"fs.azure.account.oauth2.client.secret": '#####',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/#####/oauth2/token"}

dbutils.fs.mount(
source = "abfss://netflix-movie-data@netflixmoviedata.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/netflixmovies",
extra_configs = configs)


Out[1]: True

In [0]:
%fs
ls "mnt/netflixmovies"

path,name,size,modificationTime
dbfs:/mnt/netflixmovies/raw-data/,raw-data/,0,1728304700000
dbfs:/mnt/netflixmovies/transformed-data/,transformed-data/,0,1728304713000


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
netflix_titles = spark.read.format("csv").option("header", "true").load("/mnt/netflixmovies/raw-data/netflix_titles.csv")

In [0]:
netflix_titles.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [0]:
netflix_titles.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [0]:
# First check NA values.

null_counts = netflix_titles.select([sum(col(c).isNull().cast("int")).alias(c) for c in netflix_titles.columns])
null_counts.show()

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   1|    2|    2636| 826|    832|        13|           2|     6|       5|        3|          3|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [0]:
# Let's compare how many TV shows and movies are present in the dataset.

movies_vs_shows = netflix_titles.groupBy("type").count()
movies_vs_shows.show()

+-------------+-----+
|         type|count|
+-------------+-----+
|         null|    1|
|      TV Show| 2676|
|        Movie| 6131|
|William Wyler|    1|
+-------------+-----+



In [0]:
# Fixing up the valid types.

valid_types = netflix_titles.filter(netflix_titles.type.isin("TV Show", "Movie"))
movies_vs_shows = valid_types.groupBy("type").count()
movies_vs_shows.show()

+-------+-----+
|   type|count|
+-------+-----+
|TV Show| 2676|
|  Movie| 6131|
+-------+-----+



In [0]:
# Let's see the top 5 countries with the most content.

top_countries = netflix_titles.where("country IS NOT NULL") \
                            .groupBy("country") \
                            .count() \
                            .orderBy("count", ascending=False) \
                            .limit(5)
top_countries.show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 2805|
|         India|  972|
|United Kingdom|  419|
|         Japan|  245|
|   South Korea|  199|
+--------------+-----+



In [0]:
# What about the top 10 directors with the most content?

top_directors = netflix_titles.where("director IS NOT NULL") \
                            .groupBy("director") \
                            .count() \
                            .orderBy("count", ascending=False) \
                            .limit(10)
top_directors.show()

+--------------------+-----+
|            director|count|
+--------------------+-----+
|       Rajiv Chilaka|   19|
|Raúl Campos, Jan ...|   18|
|        Marcus Raboy|   16|
|         Suhas Kadav|   16|
|           Jay Karas|   14|
| Cathy Garcia-Molina|   13|
|     Youssef Chahine|   12|
|     Martin Scorsese|   12|
|         Jay Chapman|   12|
|    Steven Spielberg|   11|
+--------------------+-----+



In [0]:
# What is the distribution of ratings?

ratings_distribution = netflix_titles.where("rating IS NOT NULL") \
                                   .groupBy("rating") \
                                   .count() \
                                   .orderBy("count", ascending=False)
ratings_distribution.show()


+-----------------+-----+
|           rating|count|
+-----------------+-----+
|            TV-MA| 3195|
|            TV-14| 2158|
|            TV-PG|  862|
|                R|  796|
|            PG-13|  489|
|            TV-Y7|  334|
|             TV-Y|  307|
|               PG|  286|
|             TV-G|  220|
|               NR|   80|
|                G|   41|
|         TV-Y7-FV|    6|
|               UR|    3|
|            NC-17|    3|
|             2021|    2|
| November 1, 2020|    1|
| Shavidee Trotter|    1|
|    Adriane Lenox|    1|
|    Maury Chaykin|    1|
|             2019|    1|
+-----------------+-----+
only showing top 20 rows



In [0]:
# Fixing up the valid ratings.

valid_ratings = ["TV-MA", "TV-14", "TV-PG", "R", "PG-13", "TV-Y7", "TV-Y", 
                 "PG", "TV-G", "NR", "G", "TV-Y7-FV", "UR", "NC-17"]
cleaned_ratings = netflix_titles.filter(netflix_titles.rating.isin(valid_ratings))

ratings_distribution = cleaned_ratings.groupBy("rating").count().orderBy("count", ascending=False)
ratings_distribution.show()

+--------+-----+
|  rating|count|
+--------+-----+
|   TV-MA| 3195|
|   TV-14| 2158|
|   TV-PG|  862|
|       R|  796|
|   PG-13|  489|
|   TV-Y7|  334|
|    TV-Y|  307|
|      PG|  286|
|    TV-G|  220|
|      NR|   80|
|       G|   41|
|TV-Y7-FV|    6|
|      UR|    3|
|   NC-17|    3|
+--------+-----+



In [0]:
# What are the top genres by personality?

top_genres = netflix_titles.groupBy("listed_in") \
                         .count() \
                         .orderBy("count", ascending=False) \
                         .limit(10)
top_genres.show()

+--------------------+-----+
|           listed_in|count|
+--------------------+-----+
|Dramas, Internati...|  361|
|       Documentaries|  358|
|     Stand-Up Comedy|  334|
|Comedies, Dramas,...|  273|
|Dramas, Independe...|  252|
|            Kids' TV|  220|
|Children & Family...|  215|
|Children & Family...|  201|
|Documentaries, In...|  186|
|Dramas, Internati...|  180|
+--------------------+-----+



In [0]:
# Let's see how much content is added per year.

netflix_titles_with_dates = netflix_titles.withColumn("date_added", to_date("date_added", "MMMM d, yyyy"))

content_per_year = netflix_titles_with_dates.where("date_added IS NOT NULL") \
                               .groupBy(year("date_added").alias("year")) \
                               .count() \
                               .orderBy("year")
content_per_year.show()

+----+-----+
|year|count|
+----+-----+
|2008|    2|
|2009|    2|
|2010|    1|
|2011|   13|
|2012|    3|
|2013|   10|
|2014|   23|
|2015|   72|
|2016|  418|
|2017| 1162|
|2018| 1623|
|2019| 1997|
|2020| 1872|
|2021| 1491|
+----+-----+



In [0]:
# What are the most common durations for movies?

common_movie_durations = netflix_titles.where(netflix_titles.type == 'Movie') \
                                     .groupBy("duration") \
                                     .count() \
                                     .orderBy("count", ascending=False)
common_movie_durations.show()

+--------+-----+
|duration|count|
+--------+-----+
|  90 min|  152|
|  94 min|  146|
|  93 min|  145|
|  97 min|  145|
|  91 min|  144|
|  95 min|  137|
|  96 min|  130|
|  92 min|  129|
| 102 min|  122|
|  98 min|  120|
|  99 min|  118|
|  88 min|  116|
| 101 min|  116|
| 103 min|  114|
| 106 min|  111|
| 100 min|  108|
|  89 min|  105|
| 104 min|  104|
|  86 min|  103|
| 105 min|  101|
+--------+-----+
only showing top 20 rows



In [0]:
# What about for TV shows?

common_movie_durations = netflix_titles.where(netflix_titles.type == 'TV Show') \
                                     .groupBy("duration") \
                                     .count() \
                                     .orderBy("count", ascending=False)
common_movie_durations.show()

+-----------------+-----+
|         duration|count|
+-----------------+-----+
|         1 Season| 1791|
|        2 Seasons|  424|
|        3 Seasons|  199|
|        4 Seasons|   95|
|        5 Seasons|   65|
|        6 Seasons|   33|
|        7 Seasons|   23|
|        8 Seasons|   17|
|        9 Seasons|    9|
|       10 Seasons|    7|
|       13 Seasons|    3|
|       11 Seasons|    2|
|       15 Seasons|    2|
|       12 Seasons|    2|
| Donnell Rawlings|    1|
|       17 Seasons|    1|
|     Bimbo Manuel|    1|
|            TV-14|    1|
+-----------------+-----+



In [0]:
# Fixing the valid durations.

valid_durations = netflix_titles.where((col("type") == "TV Show") & col("duration").rlike(r"^\d+ Season[s]?$"))

common_tv_show_durations = valid_durations.groupBy("duration") \
                                          .count() \
                                          .orderBy("count", ascending=False)
common_tv_show_durations.show()

+----------+-----+
|  duration|count|
+----------+-----+
|  1 Season| 1791|
| 2 Seasons|  424|
| 3 Seasons|  199|
| 4 Seasons|   95|
| 5 Seasons|   65|
| 6 Seasons|   33|
| 7 Seasons|   23|
| 8 Seasons|   17|
| 9 Seasons|    9|
|10 Seasons|    7|
|13 Seasons|    3|
|11 Seasons|    2|
|15 Seasons|    2|
|12 Seasons|    2|
|17 Seasons|    1|
+----------+-----+



In [0]:
netflix_titles_with_dates = netflix_titles.withColumn("date_added", to_date("date_added", "MMMM d, yyyy")).show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

In [0]:
netflix_titles_with_dates = netflix_titles.withColumn("date_added", to_date("date_added", "MMMM d, yyyy"))
netflix_titles_with_dates.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [0]:
# Let's use this simple transformation.

netflix_titles_with_dates.write.option("header", "true").csv("/mnt/netflixmovies/transformed-data/netflix_titles")